In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date_old_structure, Irr_old_structure, chart_dates, chart_labels, chart_colour, portfolio_at_date_old_structure, calculate_security_nav, currency_format_old_structure, calculate_portfolio_value_old_structure
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 23.972000000000 on 2020-12-01>

In [4]:
from django.db.models import Count

duplicates = (
            Prices.objects.values('date', 'security', 'price')
            .annotate(count_id=Count('id'))
            .filter(count_id__gt=1)
        )

total_duplicates = len(duplicates)
print((f"Found {total_duplicates} duplicate entries."))

for i, duplicate in enumerate(duplicates, start=1):
            # Keep one entry and delete the rest
            entries = Prices.objects.filter(
                date=duplicate['date'],
                security=duplicate['security'],
                price=duplicate['price']
            )
            entries_to_keep = entries.first()
            entries.exclude(id=entries_to_keep.id).delete()

            # Print status update
            print(f"Duplicates are being removed: {i} out of {total_duplicates}\r", end='', flush=True)
            
print("\nDuplicates removed.")


Found 0 duplicate entries.

Duplicates removed.


In [5]:
import datetime
from decimal import Decimal


def end_of_year_price_correction(user, year, broker_name, target_nav, asset_name):
    
    target_nav = round(Decimal(target_nav), 2)
    
    # Get the broker
    try:
        broker = Brokers.objects.get(name=broker_name)
    except Brokers.DoesNotExist:
        return {"error": f"Broker {broker_name} does not exist."}

    # Get the asset
    try:
        asset = Assets.objects.get(name=asset_name)
    except Assets.DoesNotExist:
        return {"error": f"Asset {asset_name} does not exist."}

    # Calculate end of year date
    end_of_year_date = datetime.date(year, 12, 31)

    # Fetch NAV at the end of the year
    # nav_at_end_of_year = get_nav_at_date(broker, end_of_year_date)
    nav_at_end_of_year = NAV_at_date_old_structure(user.id, [broker.id], end_of_year_date, asset.currency, [])['Total NAV']
    if nav_at_end_of_year is None:
        return {"error": f"No NAV found for broker {broker_name} at the end of {year}."}

    # Fetch asset price and position at the end of the year
    price_at_end_of_year = asset.price_at_date(end_of_year_date)
    if not price_at_end_of_year:
        return {"error": f"No price found for asset {asset_name} at the end of {year}."}

    position_at_end_of_year = asset.position(end_of_year_date, [broker.id])

    # Calculate new price
    old_price = price_at_end_of_year.price
    new_price = old_price + ((target_nav - nav_at_end_of_year) / position_at_end_of_year)

    # Display information
    old_asset_value = round(Decimal(old_price * position_at_end_of_year), 2)
    new_asset_value = round(Decimal(new_price * position_at_end_of_year), 2)

    result = {
        "old_price": old_price,
        "new_price": new_price,
        "old_asset_value": old_asset_value,
        "new_asset_value": new_asset_value,
        "old_nav": nav_at_end_of_year,
        "target_nav": target_nav
    }

    print(f"Old Price: {result['old_price']}, New Price: {result['new_price']}")
    print(f"Old Asset Value: {result['old_asset_value']}, New Asset Value: {result['new_asset_value']}")
    print(f"Old NAV: {result['old_nav']}, Target NAV: {result['target_nav']}")

    # Ask for confirmation
    confirm = input("Do you want to update the price? (yes/no): ")

    if confirm.lower() == 'yes':
        # Update the price
        price_instance, created = Prices.objects.get_or_create(
            security=asset, date=end_of_year_date,
            defaults={'price': new_price}
        )
        if not created:
            price_instance.price = new_price
            price_instance.save()

        result["status"] = "Price updated successfully."
        print("New price is:", price_instance.price)
    else:
        result["status"] = "Price update canceled."

    return result

user = CustomUser.objects.filter(username='Yaroslav').first()
year = 2013
broker_name = 'UBS Pension'
target_nav = 29444.38
asset_name = 'Emerging Markets Equity Fund'

end_of_year_price_correction(user, year, broker_name, target_nav, asset_name)

Old Price: 0.936630, New Price: 0.9366287795154500008662569358
Old Asset Value: 7983.27, New Asset Value: 7983.26
Old NAV: 29444.390402671861, Target NAV: 29444.38


{'old_price': Decimal('0.936630'),
 'new_price': Decimal('0.9366287795154500008662569358'),
 'old_asset_value': Decimal('7983.27'),
 'new_asset_value': Decimal('7983.26'),
 'old_nav': Decimal('29444.390402671861'),
 'target_nav': Decimal('29444.38'),
 'status': 'Price update canceled.'}

In [6]:
from utils import import_asset_prices_from_csv

# file_path = "../Data/Pricing data from Galaxy.csv"
investor_id = 2

# import_asset_prices_from_csv(file_path, investor_id)

In [7]:
import pandas as pd
from fuzzywuzzy import process
from datetime import datetime
from django.db.models import Q
import sys

import constants

def parse_charles_stanley_transactions(file, currency, broker_id, investor_id):
    df = pd.read_excel(file, header=3)
    transactions = []
    security_cache = {}
    skipped_count = 0

    broker = Brokers.objects.get(id=broker_id)
    investor = CustomUser.objects.get(id=investor_id)

    quantity_field = Transactions._meta.get_field('quantity')
    quantity_decimal_places = quantity_field.decimal_places
    # price_field = Transactions._meta.get_field('price')
    price_decimal_places = 5

    def find_or_prompt_security(stock_description, investor, broker, security_cache):

        # Check if we have a cached mapping
        if stock_description in security_cache:
            return security_cache[stock_description]
        
        def find_best_matching_security(description):
            securities = Assets.objects.filter(
                Q(investor=investor) & Q(brokers=broker)
            )
            security_names = [security.name for security in securities]
            best_match = process.extractOne(description, security_names, score_cutoff=60)
            
            if best_match:
                match_name, match_score = best_match
                print(f"Potential match found: '{match_name}' (Similarity: {match_score}%)")
                sys.stdout.flush()
                user_confirm = input(f"Do you agree with this match for '{description}'? (yes/no/skip/exit): ").lower()
                
                if user_confirm == 'yes':
                    matched_security = securities.get(name=match_name)
                    security_cache[stock_description] = matched_security  # Update cache
                    print(f"Match confirmed and cached for future use.")
                    return matched_security
                elif user_confirm == 'exit':
                    raise KeyboardInterrupt("User requested to exit")
                elif user_confirm == 'skip':
                    print("Skipping this transaction.")
                    security_cache[stock_description] = None  # Cache the skip decision
                    return None
                else:
                    print("Match rejected.")
                    return None
            return None

        security = find_best_matching_security(stock_description)
        
        if security is None:
            print(f"No matching security found for '{stock_description}'.")
            sys.stdout.flush()  # Ensure print is displayed
            user_input = input("Please enter the correct security name, 'skip' to skip this transaction, or 'exit' to stop processing: ")
            
            if user_input.lower() == 'exit':
                raise KeyboardInterrupt("User requested to exit")
            elif user_input.lower() == 'skip':
                print("Skipping this transaction.")
                security_cache[stock_description] = None
            elif user_input:
                # Check for an exact match with the user input
                try:
                    security = Assets.objects.get(
                        Q(investor=investor) & Q(brokers=broker) & Q(name__iexact=user_input)
                    )
                    print(f"Security '{security.name}' found and selected.")
                     # Cache the user-defined mapping
                    security_cache[stock_description] = security
                except Assets.DoesNotExist:
                    print(f"No exact match found for '{user_input}'. This transaction will be skipped.")
                    security_cache[stock_description] = None
            else:
                print("No security name provided. This transaction will be skipped.")
                security_cache[stock_description] = None

        return security
    
    def transaction_exists(transaction_data):
        return Transactions.objects.filter(
            Q(investor=transaction_data['investor']) &
            Q(broker=transaction_data['broker']) &
            Q(security=transaction_data.get('security')) &
            Q(currency=transaction_data['currency']) &
            Q(type=transaction_data['type']) &
            Q(date=transaction_data['date']) &
            Q(quantity=transaction_data.get('quantity')) &
            Q(price=transaction_data.get('price')) &
            Q(cash_flow=transaction_data.get('cash_flow')) &
            Q(commission=transaction_data.get('commission'))
            ).exists()

    for index, row in df.iterrows():
        
        date_str = row['Date']

        # Check if there is any data in the current row
        if pd.notna(date_str):
            date = datetime.strptime(date_str, '%d-%b-%Y').date()
            description = row['Description']
            stock_description = row['Stock Description']
            price = row['Price']
            debit = row['Debit']
            credit = row['Credit']

            if description == '* BALANCE B/F *' or description == 'Cash Transfers ISA':
                print(f'Skipped: {description}')
                continue

            elif "Gross interest" in description:
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_INTEREST_INCOME,
                    'date': date,
                    'cash_flow': round(Decimal(str(credit)), 2),
                }

            elif description == 'Funds Platform Fee' or description == 'Govt Flat Rate Int Charge':
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_BROKER_COMMISSION,
                    'date': date,
                    'commission': round(-Decimal(str(debit)), 2),
                }

            elif description == 'Stocks & Shares Subs' or 'ISA Subscription' in description:
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_CASH_IN,
                    'date': date,
                    'cash_flow': Decimal(str(credit)),
                }

            elif 'Dividend' in description or 'Equalisation' in description or 'Tax Credit' in description:
                security = find_or_prompt_security(stock_description, investor, broker, security_cache)
                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'security': security,
                    'currency': currency,
                    'type': constants.TRANSACTION_TYPE_DIVIDEND,
                    'date': date,
                    'cash_flow': Decimal(str(credit)),
                }

            elif pd.notna(stock_description):

                security = find_or_prompt_security(stock_description, investor, broker, security_cache)
                if security is None:
                    continue # Skip this transaction

                transaction_type = constants.TRANSACTION_TYPE_BUY if debit > 0 else constants.TRANSACTION_TYPE_SELL
                price = round(Decimal(str(price)), price_decimal_places)
                if transaction_type == constants.TRANSACTION_TYPE_BUY:
                    quantity = round(Decimal(str(debit)) / price, quantity_decimal_places)
                else:
                    quantity = round(-Decimal(str(credit)) / price, quantity_decimal_places)

                transaction_data = {
                    'investor': investor,
                    'broker': broker,
                    'security': security,
                    'currency': currency,
                    'type': transaction_type,
                    'date': date,
                    'quantity': quantity,
                    'price': price,
                }

            else:
                print(f'Skipped: {description}')
                continue  # Skip if not a recognized transaction type

            if transaction_exists(transaction_data):
                print(f"Skipping existing transaction: {transaction_data}")
                skipped_count += 1
                continue

            transactions.append(transaction_data)

    print(f"\nProcessed {len(transactions)} transactions.")
    print(f"Skipped {skipped_count} existing transactions.")

    if transactions:
        save_choice = input(f"Do you want to save these transactions for {broker.name}? (yes/no): ").lower()
        if save_choice == 'yes':
            Transactions.objects.bulk_create([Transactions(**data) for data in transactions])
            print("Transactions saved to the database.")
        else:
            print("Transactions were not saved to the database.")
    else:
        print("No transactions to save.")

    return transactions

file = "../Data/Statement_ISA_4681921_Generated_2158_08Aug.xlsx"
currency = 'GBP'
broker_id = 10
investor_id = 2

t = parse_charles_stanley_transactions(file, currency, broker_id, investor_id)

Skipping existing transaction: {'investor': <CustomUser: Yaroslav>, 'broker': <Brokers: Charles Stanley – ISA>, 'currency': 'GBP', 'type': 'Broker commission', 'date': datetime.date(2024, 8, 1), 'commission': Decimal('-30.55')}
Skipping existing transaction: {'investor': <CustomUser: Yaroslav>, 'broker': <Brokers: Charles Stanley – ISA>, 'currency': 'GBP', 'type': 'Broker commission', 'date': datetime.date(2024, 7, 1), 'commission': Decimal('-29.33')}
Potential match found: 'Legal & General Multi-Index' (Similarity: 86%)
Match rejected.
No matching security found for 'LEGAL & GENERAL UT L&G GBL TECH IDX TRUST I AC'.
No security name provided. This transaction will be skipped.
Skipping existing transaction: {'investor': <CustomUser: Yaroslav>, 'broker': <Brokers: Charles Stanley – ISA>, 'currency': 'GBP', 'type': 'Broker commission', 'date': datetime.date(2024, 6, 3), 'commission': Decimal('-29.62')}
Potential match found: 'Rathbone Ethical Bond' (Similarity: 90%)


KeyboardInterrupt: User requested to exit

In [22]:
from datetime import timedelta
from typing import Any, List, Optional, Dict, Tuple
from decimal import Decimal

from core.portfolio_utils import IRR, get_fx_rate


def calculate_closed_table_output_for_api(
    user_id: int,
    portfolio: List[Any],
    end_date: date,
    categories: List[str],
    use_default_currency: bool,
    currency_target: str,
    selected_brokers: List[int],
    start_date: Optional[date] = None
) -> Tuple[List[Dict[str, Any]], Dict[str, Any]]:
    """
    Calculate table output for closed positions.

    :param user_id: The ID of the user
    :param portfolio: List of asset objects
    :param end_date: The end date for calculations
    :param categories: List of categories to include in the output
    :param use_default_currency: Whether to use the default currency
    :param currency_target: The target currency for calculations
    :param selected_brokers: List of selected broker IDs
    :param start_date: The start date for calculations (optional)
    :return: A tuple containing a list of closed position dictionaries and a dictionary of totals
    """
    closed_positions = []
    totals = ['entry_value', 'current_value', 'realized_gl', 'capital_distribution', 'commission']
    portfolio_closed_totals = {}

    # print(list((set(totals) & set(categories)) | {'entry_value', 'exit_value', 'total_return_amount', 'price_change_percentage', 'capital_distribution_percentage', 'commission_percentage', 'total_return_percentage'}))
    
    for asset in portfolio:

        print(asset)

        
        
        exit_dates = list(asset.exit_dates(end_date, selected_brokers, start_date))
        entry_dates = list(asset.entry_dates(end_date, selected_brokers))

        print(exit_dates, entry_dates)
        
        for i, exit_date in enumerate(exit_dates):
            
            position = {
                'type': asset.type,
                'name': asset.name,
                'currency': currency_format_old_structure(None, asset.currency)
            }
            
            currency_used = None if use_default_currency else currency_target
            
            position['exit_date'] = exit_date

            # Determine entry_date
            first_entry_date = asset.entry_dates(exit_date, selected_brokers)[-1]
            entry_date = start_date if start_date and start_date >= first_entry_date else first_entry_date
            print(exit_date, first_entry_date, entry_date)
            position['investment_date'] = entry_date

            # Determine next_entry_date (or end_date if there's no next entry)
            next_entry_date = entry_dates[entry_dates.index(entry_date) + 1] if entry_date in entry_dates and entry_dates.index(entry_date) < len(entry_dates) - 1 else end_date

            asset_transactions = asset.transactions.filter(
                investor__id=user_id,
                date__gte=entry_date,
                date__lte=exit_date,
                broker__in=selected_brokers,
                quantity__isnull=False
            ).order_by('-date')

            # Determine if it's a long or short position
            is_long_position = asset_transactions.first().quantity < 0 # if asset_transactions.exists() else True

            if is_long_position:
                entry_transactions = asset_transactions.filter(quantity__gt=0)
                exit_transactions = asset_transactions.filter(quantity__lt=0)
            else:
                entry_transactions = asset_transactions.filter(quantity__lt=0)
                exit_transactions = asset_transactions.filter(quantity__gt=0)

            # Calculate entry value and quantity
            if start_date is not None:
                entry_quantity = asset.position(entry_date - timedelta(days=1), selected_brokers)
                entry_value = asset.price_at_date(entry_date - timedelta(days=1), currency_used).price * entry_quantity
                print(asset.position(entry_date - timedelta(days=1), selected_brokers))
            else:
                entry_value = Decimal(0)
                entry_quantity = Decimal(0)
            
            for transaction in entry_transactions:
                fx_rate = get_fx_rate(transaction.currency, currency_used, transaction.date) if currency_used else 1
                entry_value += transaction.price * abs(transaction.quantity) * fx_rate
                entry_quantity += abs(transaction.quantity)

            position['entry_value'] = Decimal(entry_value)

            # Calculate exit value and quantity
            exit_value = Decimal(0)
            for transaction in exit_transactions:
                fx_rate = get_fx_rate(transaction.currency, currency_used, transaction.date) if currency_used else 1
                exit_value += transaction.price * abs(transaction.quantity) * fx_rate

            position['exit_value'] = Decimal(exit_value)

            # Calculate realized gain/loss
            if 'realized_gl' in categories:
                position['realized_gl'] = exit_value - entry_value
            else:
                position['realized_gl'] = Decimal(0)

            position['price_change_percentage'] = (position['realized_gl']) / position['entry_value'] if position['entry_value'] > 0 else 'N/R'

            # Calculate capital distribution including dividends after exit_date but before next_entry_date
            if 'capital_distribution' in categories:
                position['capital_distribution'] = (
                    asset.get_capital_distribution(exit_date, currency_used, selected_brokers, entry_date) +
                    asset.get_capital_distribution(next_entry_date, currency_used, selected_brokers, exit_date + timedelta(days=1))
                )
                position['capital_distribution_percentage'] = Decimal(position['capital_distribution'] / position['entry_value']) if position['entry_value'] > 0 else 'N/R'
            else:
                position['capital_distribution'] = Decimal(0)

            if 'commission' in categories:
                position['commission'] = asset.get_commission(exit_date, currency_used, selected_brokers, entry_date)
                position['commission_percentage'] = position['commission'] / position['entry_value'] if position['entry_value'] > 0 else 'N/R'
            else:
                position['commission'] = Decimal(0)

            position['total_return_amount'] = position['realized_gl'] + position['capital_distribution'] + position['commission']
            position['total_return_percentage'] = position['total_return_amount'] / position['entry_value'] if position['entry_value'] > 0 else 'N/R'

            # Calculate IRR
            currency_used = asset.currency if use_default_currency else currency_target
            position['irr'] = IRR(user_id, exit_date, currency_used, asset_id=asset.id, broker_id_list=selected_brokers, start_date=entry_date)

            closed_positions.append(position)
            
            # Update portfolio totals
            for key in list((set(totals) & set(categories)) | {'entry_value', 'exit_value', 'total_return_amount', 'price_change_percentage', 'capital_distribution_percentage', 'commission_percentage', 'total_return_percentage'}):
                if key in position:
                    # print(f"Key: {key}, Position: {position[key]}")
                    portfolio_closed_totals[key] = portfolio_closed_totals.get(key, Decimal(0)) + position[key]
                    # print(f"Portfolio Closed Totals: {portfolio_closed_totals[key]}")



        # # Update totals with raw values
        # for key in (list(set(totals) & set(categories)) + ['entry_value', 'exit_value', 'total_return_amount']):
        #     if key in position:
        #         portfolio_closed_totals[key] = portfolio_closed_totals.get(key, 0) + position[key]
    
    # Calculate percentage totals
    if portfolio_closed_totals.get('entry_value', Decimal(0)) != 0:
        portfolio_closed_totals['price_change_percentage'] = (portfolio_closed_totals.get('realized_gl', Decimal(0)) + portfolio_closed_totals.get('unrealized_gl', Decimal(0))) / abs(portfolio_closed_totals['entry_value'])
        if 'capital_distribution' in categories:
            portfolio_closed_totals['capital_distribution_percentage'] = portfolio_closed_totals['capital_distribution'] / portfolio_closed_totals['entry_value']
        if 'commission' in categories:
            portfolio_closed_totals['commission_percentage'] = portfolio_closed_totals['commission'] / portfolio_closed_totals['entry_value']
        portfolio_closed_totals['total_return_percentage'] = portfolio_closed_totals['total_return_amount'] / abs(portfolio_closed_totals['entry_value'])
    
    return closed_positions, portfolio_closed_totals

user = CustomUser.objects.get(username='Yaroslav')
portfolio = Assets.objects.filter(investor=user, name='UBS Glo ex-UK C')
end_date = date(2011, 12, 31)
categories = ['investment_date', 'realized_gl', 'capital_distribution', 'commission'] + ['exit_date']
use_default_currency = True
currency_target = 'GBP'
# selected_brokers = None
# start_date = date(2011, 1, 1)

brokers = Brokers.objects.filter(investor=user, name='UBS Pension')
selected_brokers = [broker.id for broker in brokers]

closed_positions, portfolio_closed_totals = calculate_closed_table_output_for_api(
    user.id,
    portfolio,
    end_date,
    categories,
    use_default_currency,
    currency_target,
    selected_brokers,
    start_date = None
)

print(closed_positions)
# print(portfolio_closed_totals)

UBS Glo ex-UK C
[datetime.date(2011, 3, 24), datetime.date(2011, 5, 16)] [datetime.date(2010, 3, 3), datetime.date(2011, 3, 25)]
2011-03-24 2010-03-03 2010-03-03
2011-05-16 2011-03-25 2011-03-25
[{'type': 'Mutual fund', 'name': 'UBS Glo ex-UK C', 'currency': '£', 'exit_date': datetime.date(2011, 3, 24), 'investment_date': datetime.date(2010, 3, 3), 'entry_value': Decimal('1036.524978650750'), 'exit_value': Decimal('1121.784941030000'), 'realized_gl': Decimal('85.259962379250'), 'price_change_percentage': Decimal('0.08225557910841023832539232587'), 'capital_distribution': Decimal('0'), 'capital_distribution_percentage': Decimal('0E+12'), 'commission': Decimal('0'), 'commission_percentage': Decimal('0E+12'), 'total_return_amount': Decimal('85.259962379250'), 'total_return_percentage': Decimal('0.08225557910841023832539232587'), 'irr': Decimal('0.0975')}, {'type': 'Mutual fund', 'name': 'UBS Glo ex-UK C', 'currency': '£', 'exit_date': datetime.date(2011, 5, 16), 'investment_date': datetim